# Tableau Project (Criterio 1: df_capability)

In [4]:
#Tras iniciar servidor de DB mongo, importo módulo 'pymongo'.
import pymongo
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize
#Asigno variable al cliente de DB y a la DB.
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies
cursor = db.companies.find()
data = list(cursor)
df = pd.DataFrame(data)

In [5]:
col_float = ['deadpooled_day','deadpooled_month','deadpooled_year','founded_day','founded_month',
             'founded_year','number_of_employees']
col_obj = ['_id','acquisition','acquisitions','alias_list','blog_feed_url','blog_url','category_code',
           'competitions','created_at','crunchbase_url','deadpooled_url','description','email_address',
        'external_links','funding_rounds','homepage_url','image','investments','ipo','milestones', 
           'name','offices','overview','partners','permalink','phone_number','products','providerships',
        'relationships','screenshots','tag_list','total_money_raised','twitter_username','updated_at',
           'video_embeds']
col_drop1 = ['deadpooled_day','deadpooled_month','founded_day','founded_month','alias_list','blog_feed_url',
             'blog_url','created_at','crunchbase_url','deadpooled_url','email_address','external_links',
            'image','permalink','phone_number','screenshots','tag_list','updated_at','video_embeds']
col_drop2 = ['acquisition','acquisitions','competitions','description','funding_rounds','homepage_url',
             'investments','ipo','milestones','partners','products','providerships','relationships',
             'twitter_username','overview',]
col_ok = ['deadpooled_year','founded_year','number_of_employees','_id','category_code','name','offices',
          'total_money_raised']

In [10]:
#Elimino columnas con valores que no me interesan
df_drop1 = df.drop(col_drop1, axis=1)
df_drop2 = df_drop1.drop(col_drop2, axis=1)
#Elijo solo las empresas con deadpooled_year == null
df_deadnull = df_drop2[df_drop2['deadpooled_year'].isnull()]
#Elimino filas con valores nulos de las columnas que me interesan
df_nulls = df_deadnull.dropna(subset=['founded_year','number_of_employees','name','offices',
          'total_money_raised'])
display(len(df_nulls))

7934

In [11]:
#Data frames por tipo de empresas y elimino las columnas _id y deadpooled_year 
col_cat = ['games_video']
df_cat = df_nulls[df_nulls['category_code'].isin(col_cat)]\
.drop(['_id','deadpooled_year'], axis=1)
print(len(df_cat))

525


In [12]:
#Defino funciones para la modificación de los datos de las columnas a analizar.

#Transforma un dato tipo float a uno tipo int.
def float_to_int(flt):
    integer = int(flt)
    return integer

#Transforma un dato tipo str en int devolviendo solo el número.
def str_to_int(strg):
    string = re.findall('\d+', strg )
    integer = int(string[0])*1000
    return integer


In [14]:
#Elimino registros irrelevantes (tomando solo los registros que me interesan)
df_regout = df_cat.copy()
df_regout['total_money_raised'] = df_regout['total_money_raised'].apply(str_to_int)
df_regout['founded_year'] = df_regout['founded_year'].apply(float_to_int)
#Elimino registros según criterio 1: solo empresas de video juegos que hayan levantado capital 
#(me da igual el número de empleados)
df_regout = df_regout[df_regout.total_money_raised != 0]
#df_regout = df_regout[df_regout.number_of_employees != 0]
df_regout = df_regout[df_regout['offices'].map(len) > 0]
print(len(df_regout))

128


In [15]:
#Transforma los datos de ubicación de las oficinas en formato GeoJson y selecciona solo la primera ubicación
def nested_to_list(dicts):
    lat_lon = [{"type":"Point","coordinates":[l['longitude'],l['latitude']]} for l in dicts]
    lst = lat_lon[0]
    long = lst['coordinates'][0]
    if long != None:
        return lst
    else:
        return 0


In [16]:
#Transformo valores de columnas aplicando las funciones anteriormente definidas.
df_colout = df_regout.copy()
df_colout['offices'] = df_colout['offices'].apply(nested_to_list)
print(len(df_colout))

128


In [17]:
#Elimino valores de GeoJson nulos
df_geoout = df_colout.copy()
df_geoout = df_geoout[df_geoout.offices != 0]
print(len(df_geoout))
display(df_geoout.head())

99


,category_code,founded_year,name,number_of_employees,offices,total_money_raised
16,games_video,2006,Joost,0.0,"{'type': 'Point', 'coordinates': [-74.0094471,...",45000
31,games_video,2006,Kyte,40.0,"{'type': 'Point', 'coordinates': [-122.409173,...",23000
51,games_video,2005,YouTube,0.0,"{'type': 'Point', 'coordinates': [-122.426804,...",11000
52,games_video,2007,Livestream,120.0,"{'type': 'Point', 'coordinates': [-73.995625, ...",14000
54,games_video,2007,AdaptiveBlue,15.0,"{'type': 'Point', 'coordinates': [-74.3372, 40...",24000


In [18]:
df_geoout.to_json('capability.json', orient="records", lines=True)

### Import mongodb y creo indice para función $near
Nombre de la collection: **capability**

In [31]:
#Creo función $near
def concentration(company):
    cursor_near = db.capability.find({
      "offices": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [company['offices']['coordinates'][0], company['offices']['coordinates'][1]]
          },
          "$minDistance": 0,
          "$maxDistance":10000
        }
      }
    })
    return list(cursor_near)

In [32]:
#Hago loop sobre las empresas
cursor_geojson = db.capability.find()
clusters = []
for element in cursor_geojson:
    clusters.append(len(concentration(element)))

display(len(clusters))
print(clusters)
print(max(clusters))

99

[12, 12, 10, 2, 1, 6, 12, 10, 5, 6, 10, 4, 1, 4, 10, 3, 12, 10, 10, 3, 4, 3, 3, 1, 3, 1, 3, 1, 10, 1, 1, 7, 3, 12, 1, 3, 1, 3, 7, 1, 1, 2, 1, 3, 7, 1, 4, 6, 10, 12, 3, 3, 3, 1, 12, 1, 12, 12, 3, 3, 10, 1, 6, 1, 3, 2, 6, 3, 1, 3, 10, 2, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 12, 3, 2, 3, 3, 1, 4, 2, 1, 12, 6, 1, 6, 6, 1, 12, 1]
12


In [33]:
target = list(db.capability.find())
targets = []
for i in range(len(clusters)):
    if clusters[i] == max(clusters):
        targets.append([target[i]['offices']['coordinates'][0], target[i]['offices']['coordinates'][1]])

print(targets)

[[-74.0094471, 40.7464969], [-73.995625, 40.726155], [-73.99873, 40.722655], [-74.001704, 40.723384], [-74.002662, 40.717248], [-73.9930489, 40.73993], [-73.957968, 40.720764], [-73.988543, 40.754762], [-73.998726, 40.738476], [-73.97593, 40.752672], [-73.98618, 40.743877], [-74.005568, 40.7523796]]


In [40]:
#Dataframe para plateau
long_lat = ['Longitude','Latitude']
df_capability = pd.DataFrame(targets, columns=long_lat)
df_capability['Criteria'] = 'Capability'
display(df_capability)

,Longitude,Latitude,Criteria
0,-74.009447,40.746497,Capability
1,-73.995625,40.726155,Capability
2,-73.998730,40.722655,Capability
3,-74.001704,40.723384,Capability
4,-74.002662,40.717248,Capability
5,-73.993049,40.739930,Capability
6,-73.957968,40.720764,Capability
7,-73.988543,40.754762,Capability
8,-73.998726,40.738476,Capability
9,-73.975930,40.752672,Capability


In [41]:
df_capability.to_json('df_capability.json', orient="records", lines=True)